In [1]:
import cv2, os, numpy as np
import time
import pyautogui
from PIL import ImageGrab
from typing import List

In [2]:
from os import listdir
from os.path import isfile
mypath = "C:/Users/Admin/Desktop/Code/wsop/WSOP-daily-bliz/card/"
cardfile = [mypath+f for f in listdir(mypath)]
mypath = "C:/Users/Admin/Desktop/Code/wsop/WSOP-daily-bliz/phototest/"
imagefile = [mypath+f for f in listdir(mypath) if len(f)==5 or len(f)==6]

In [3]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [4]:
def value(use: List[str]):
    '''
    code that change card(str) -> card(int)
    and check it flower because only flush, straight flush and royal straight flush that use flower
    '''
    for i in use:
        i=i.strip('.png')
        
    dic={ #change str to int easily
        "1":14,
        "2":2,
        "3":3,
        "4":4,
        "5":5,
        "6":6,
        "7":7,
        "8":8,
        "9":9,
        "T":10,
        "J":11,
        "Q":12,
        "K":13,
    }
    
    temp=[]
    flowers=0
    flushed=1
    if(len(use)>0):
        trash,flowers=use[0].split('-')
    for i in use:
        number,flower = i.split('-')
        if(flower!=flowers): #check flush here for flush and straight flush
            flushed=0
        temp.append(dic[number]) #change str to int
    
    use=temp
    use.sort()

    highcard=0 #1
    onepair=0 #100
    twopair=[] #problem_here #1000
    threeofkind=0 #10000
    straight=0 #100000
    fullhouse=0 #1000000
    flush=0 #10000000
    fourofkind=0 #100000000
    straightflush=0 #1000000000
    royal=0 #0
    royaleSF=0 #10000000000
    '''
    priority is card number(Ps.(1)=14) plus with number after that line
    callulate priority in line 166 to 222
    '''

    #highcard
    for j in range(len(use)):
        highcard=max(highcard,use[j])

    #Duplicate_Card
    #onepair
    #twopair
    #threeofkind
    #fourofkind
    dupcard2=[]
    skip=0
    for j in range(len(use)):
        if(skip!=0):
            skip-=1
            continue
        #fourofkind
        if(j+3<len(use) and use[j]==use[j+1]==use[j+2]==use[j+3]):
            fourofkind=use[j]
            skip+=3
        #threeofkind
        elif(j+2<len(use) and use[j]==use[j+1]==use[j+2]):
            threeofkind=use[j]
            skip+=2
        #twopair and onepair
        elif(j+1<len(use) and use[j]==use[j+1]):
            dupcard2.append(use[j])
            skip+=1

    if(len(dupcard2)==2):
        twopair.append(dupcard2[0])
        twopair.append(dupcard2[1])
    elif(len(dupcard2)==1):
        onepair=dupcard2[0]
    
    #flush
    if(flushed==1):
        for j in range(len(use)):
            flush=max(flush,use[j])

    #fullhouse
    if(threeofkind!=0 and onepair!=0):
        fullhouse=threeofkind

    #staright
    if(len(use)==5 and use[0]==use[1]-1 and use[1]==use[2]-1 and use[2]==use[3]-1 and use[3]==use[4]-1):
        straight=use[4]
    elif(len(use)==5 and use[0]==10 and use[1]==11 and use[2]==12 and use[3]==13  and use[4]==14):
        straight=1
        royal=1
    elif(len(use)==5 and use[0]==2 and use[1]==3 and use[2]==4 and use[3]==5 and use[4]==14):
        straight=2

    if(flush!=0 and straight!=0):
        if(royal==1):
            royalSF=1 #royalstrightflush
        else:
            straightflush=straight #rstrightflush

    # highcard=0 #1
    # onepair=0 #1e2
    # twopair=0 #1e3
    # threeofkind=0 #1e4
    # straight=0 #1e5
    # flush=0 #1e6
    # fullhouse=0 #1e7
    # fourofkind=0 #1e8
    # straightflush=0 #1e9
    # royale=0 #0
    # royaleSF=0 #1e10
    
    #for return is priority(int), howwin(str), dataforsort(list), cards in you hand that u shold choose(list)#
    if(royaleSF!=0):
        return [1e10,'royaleSF',[0],use]
        #pass
    elif(straightflush!=0): 
        if(straight==1):
            straight=14
        return [1e9+straightflush,'straightflush',[0],use]
        #pass
    elif(fourofkind!=0):
        if(fourofkind==1):
            fourofkind=14
        return [1e8+fourofkind,'fourofkind',[0],use]
        #pass
    elif(fullhouse!=0): 
        if(fullhouse==1):
            fullhouse=14
        return [1e7+fullhouse,'fullhouse',[0],use]
        #pass
    elif(flush!=0): 
        if(flush==1):
            flush=14
        return [1e6+flush,'flush',use,use]
        #tie pass
    elif(straight!=0): 
        if(straight==1):
            straight=14
        return [1e5+straight,'straight',[0],use]
        #pass
    elif(threeofkind!=0): 
        if(threeofkind==1):
            threeofkind=14
        return [1e4+threeofkind,'threeofkind',[0],use]
        #pass
    elif(len(twopair)!=0):
        left = sum(use)-2*(twopair[0]+twopair[1])
        if(twopair[0]==1):
            twopair[0]=14
        return [1e3+twopair[0]+twopair[1]*100,'twopair',[left],use]
        #tie  pass
    elif(onepair!=0): 
        tmp=use.copy()
        if(onepair==1):
            onepair=14
        for j in range(len(use)):
            if(j+1<len(use) and use[j]==use[j+1]):
                use.pop(j)
                use.pop(j)
                break
        return [1e2+onepair,'onepair',use,tmp]
        #tie pass
    elif(highcard!=0): 
        if(highcard==1):
            highcard=14
        return [highcard,'highcard',use,use]

In [5]:
def best_card_ofone(card: List[str]):
    #print(card)
    best=[]
    for i in range(len(card)):
        for j in range(len(card)):
            if i < j:
                tem=[]
                for z in range(len(card)):
                    if(z!=i and z!=j):
                        tem.append(card[z])
                best.append(tem)

    val=[]
    for i in range(len(best)):
        val.append(value(best[i]))
    val.sort()
    val.reverse()
    return val[0]

In [6]:
def whoiswinnner(flop: List[str], hand1: List[str], hand2: List[str]) -> int:
    hand1 += flop
    hand2 += flop
    use=[]
    use.append(best_card_ofone(hand1)+[1])
    use.append(best_card_ofone(hand2)+[2])
    use[0].pop(2)
    use[1].pop(2)
    use.sort()
    # print(use)
    return use[1][-1]

In [7]:
# x=whoiswinnner(['5-diamond.png', 'K-club.png', 'Q-diamond.png', 'J-club.png', '8-club.png'],['1-club.png', '8-heart.png'],['1-diamond.png', 'J-spades.png'])
# print(x)

In [8]:
def show_image(image):
    cv2.imshow("result", image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [9]:
def cal(image):
    test_img = image
    
    mid_img = test_img[280:850, 420:1500]
    
    test_img = rotate_image(test_img,-10)
    left_img = test_img[540:810, 400:550]
   
    test_img = rotate_image(test_img,20)
    right_img = test_img[540:810, 1300:1600]
    
    flop=[]
    hand1=[]
    hand2=[]
    
    for card in cardfile:
        test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
        result = cv2.matchTemplate(mid_img, test_card, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        h, w = test_card.shape

        threshold = 0.96
        yloc, xloc = np.where(result >= threshold)
        for x, y in zip(xloc, yloc):
            cv2.rectangle(mid_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        rectangles = []
        for x,y in zip(xloc, yloc):
            rectangles.append([int(x), int(y), int(w), int(h)])
            rectangles.append([int(x), int(y), int(w), int(h)])
            
        rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
        if(len(rectangles)>=1):
            xloc,yloc=sum(xloc)//len(xloc) , sum(yloc)//len(yloc) 
            if(yloc<=100):
                flop.append(card.split('/')[-1])
            elif(xloc<450):
                hand1.append(card.split('/')[-1])
            else:
                hand2.append(card.split('/')[-1])
                
    for card in cardfile:
        test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
        result = cv2.matchTemplate(left_img, test_card, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        h, w = test_card.shape

        threshold = 0.96
        yloc, xloc = np.where(result >= threshold)
        for x, y in zip(xloc, yloc):
            cv2.rectangle(left_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
        rectangles = []
        for x,y in zip(xloc, yloc):
            rectangles.append([int(x), int(y), int(w), int(h)])
            rectangles.append([int(x), int(y), int(w), int(h)])
            
        rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
        if(len(rectangles)>=1):
            hand1.append(card.split('/')[-1])

    for card in cardfile:
        test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
        result = cv2.matchTemplate(right_img, test_card, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        h, w = test_card.shape

        threshold = 0.96
        yloc, xloc = np.where(result >= threshold)
        for x, y in zip(xloc, yloc):
            cv2.rectangle(right_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
        rectangles = []
        for x,y in zip(xloc, yloc):
            rectangles.append([int(x), int(y), int(w), int(h)])
            rectangles.append([int(x), int(y), int(w), int(h)])
            
        rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
        if(len(rectangles)>=1):
            hand2.append(card.split('/')[-1])
    
    if len(flop)!=5 and len(hand1)!=2 and len(hand2)!=2:
        return 0
    # print(flop)
    # print(hand1)
    # print(hand2)
    if whoiswinnner(flop,hand1,hand2)==1:
        return 1
    else:
        return 2

In [10]:
i=0
time.sleep(2)
while True:
    i+=1
    img = ImageGrab.grab(bbox=(0, 0, 1920, 1080)) #x, y, w, h
    img_np = np.array(img)
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)

    val=cal(frame)
    
    if(val==1):
        pyautogui.click(500, 700)
        time.sleep(2.1)
    elif(val==2):
        pyautogui.click(1400, 700)
        time.sleep(2.1)
    else:
        pass

KeyboardInterrupt: 

In [ ]:
# DEBUG
# for image in imagefile:
#     print("----------------------------")
#     test_img = cv2.imread(image, cv2.IMREAD_GRAYSCALE) 
    
#     mid_img = test_img[280:850, 420:1500]
    
#     test_img = rotate_image(test_img,-10)
#     left_img = test_img[540:810, 400:550]
   
#     test_img = rotate_image(test_img,20)
#     right_img = test_img[540:810, 1300:1600]
    
#     flop=[]
#     hand1=[]
#     hand2=[]
    
#     for card in cardfile:
#         test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
#         result = cv2.matchTemplate(mid_img, test_card, cv2.TM_CCOEFF_NORMED)
#         min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
#         h, w = test_card.shape

#         threshold = 0.96
#         yloc, xloc = np.where(result >= threshold)
#         for x, y in zip(xloc, yloc):
#             cv2.rectangle(mid_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
#         rectangles = []
#         for x,y in zip(xloc, yloc):
#             rectangles.append([int(x), int(y), int(w), int(h)])
#             rectangles.append([int(x), int(y), int(w), int(h)])
            
#         rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
#         if(len(rectangles)>=1):
#             xloc,yloc=sum(xloc)//len(xloc) , sum(yloc)//len(yloc) 
#             if(yloc<=100):
#                 flop.append(card.split('/')[-1])
#             elif(xloc<450):
#                 hand1.append(card.split('/')[-1])
#             else:
#                 hand2.append(card.split('/')[-1])
                
#     for card in cardfile:
#         test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
#         result = cv2.matchTemplate(left_img, test_card, cv2.TM_CCOEFF_NORMED)
#         min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
#         h, w = test_card.shape

#         threshold = 0.96
#         yloc, xloc = np.where(result >= threshold)
#         for x, y in zip(xloc, yloc):
#             cv2.rectangle(left_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
#         rectangles = []
#         for x,y in zip(xloc, yloc):
#             rectangles.append([int(x), int(y), int(w), int(h)])
#             rectangles.append([int(x), int(y), int(w), int(h)])
            
#         rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
#         if(len(rectangles)>=1):
#             hand1.append(card.split('/')[-1])

#     for card in cardfile:
#         test_card = cv2.imread(card, cv2.IMREAD_GRAYSCALE)
#         result = cv2.matchTemplate(right_img, test_card, cv2.TM_CCOEFF_NORMED)
#         min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
#         h, w = test_card.shape

#         threshold = 0.96
#         yloc, xloc = np.where(result >= threshold)
#         for x, y in zip(xloc, yloc):
#             cv2.rectangle(right_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            
#         rectangles = []
#         for x,y in zip(xloc, yloc):
#             rectangles.append([int(x), int(y), int(w), int(h)])
#             rectangles.append([int(x), int(y), int(w), int(h)])
            
#         rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.2)
#         if(len(rectangles)>=1):
#             hand2.append(card.split('/')[-1])
            
#     print(flop)
#     print(hand1)
#     print(hand2)
#     print(image.split('/')[-1])
#     if(len(hand1)!=2 or len(hand2)!=2): continue
#     if whoiswinnner(flop,hand1,hand2)==1:
#         print(1)
#     else:
#         print(2)
    